In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from WindPy import w
import pyodbc
import datetime as dt
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.0.185.138;DATABASE=JYDB;UID=js_dev;PWD=js_dev')

In [4]:
report_date = '2020-06-30'
last_date = dt.datetime.today() - dt.timedelta(1)

In [5]:
def get_wind_code(c):
    if c.startswith('0') or c.startswith('3'):
        return c + '.SZ'
    else:
        return c + '.SH'

# 提取PMS行业基金列表

In [6]:
owner = 'W5718909221'
today = dt.date.today().strftime('%Y%m%d')

### 可选消费

In [7]:
data = w.wpf("可选消费行业基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [8]:
con = pd.DataFrame(data.Data, index=data.Fields)
con = con.T

In [9]:
con = con[con['asset_classification'] == '基金']

### 医药

In [10]:
data = w.wpf("医药行业基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [11]:
med = pd.DataFrame(data.Data, index=data.Fields)
med = med.T

In [12]:
med = med[med['asset_classification'] == '基金']

### 科技

In [13]:
data = w.wpf("科技行业基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [14]:
tech = pd.DataFrame(data.Data, index=data.Fields)
tech = tech.T

In [15]:
tech = tech[tech['asset_classification'] == '基金']

### 价值

In [16]:
data = w.wpf("全市场价值基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [17]:
value = pd.DataFrame(data.Data, index=data.Fields)
value = value.T

In [18]:
value = value[value['asset_classification'] == '基金']

### 成长

In [19]:
data = w.wpf("全市场成长基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [20]:
growth = pd.DataFrame(data.Data, index=data.Fields)
growth = growth.T

In [21]:
growth = growth[growth['asset_classification'] == '基金']

### 平衡

In [22]:
data = w.wpf("全市场平衡基金组合", 
             "PMS.HoldingDaily",
             "tradedate=%s;reportcurrency=CNY;owner=%s;field=Asset_Code,Asset_Name,Asset_Classification"%(today, owner))

In [23]:
garp = pd.DataFrame(data.Data, index=data.Fields)
garp = garp.T

In [24]:
garp = garp[garp['asset_classification'] == '基金']

In [25]:
funds = pd.concat([con, med, tech, value, growth, garp])

# 提取基金重仓股

In [26]:
set_codes = '(' + ','.join(['\'%s\''%(x.rstrip('.OF')) for x in funds['asset_code']]) + ')'

In [27]:
query = """
    select ksp.ReportDate, t.SecuCode, t.SecuAbbr, t2.SecuCode, t2.SecuAbbr, ksp.SharesHolding, ksp.ChangeOfSharesHolding, ksp.RatioInNV
    from MF_KeyStockPortfolio as ksp, SecuMain as t, SecuMain as t2
    where ksp.ReportDate = '%s'
        and ksp.InnerCode = t.InnerCode
        and ksp.StockInnerCode = t2.InnerCode
        and t.SecuCode in %s
"""%(report_date, set_codes)

In [28]:
df = pd.read_sql_query(query, con=conn)

In [29]:
df.columns = ['ReportDate', 'SecuCode', 'SecuAbbr', 'StockCode', 'StockAbbr', 'SharesHolding', 'ChangeOfSharesHolding', 'RatioInNV']

In [30]:
df['StockAbbr'] = df['StockAbbr'].apply(lambda x: x.replace(' ', ''))

In [31]:
stocks = df.groupby(['StockCode', 'StockAbbr']).sum()[['SharesHolding', 'ChangeOfSharesHolding']]

In [32]:
stocks['RatioInNV'] = df.groupby(['StockCode', 'StockAbbr']).mean()['RatioInNV']

In [33]:
stocks['WindCode'] = [get_wind_code(x) for x in stocks.index.levels[0]]

In [34]:
data = w.wss(stocks['WindCode'].tolist(), 'close', "tradeDate=%s"%(last_date.strftime('%Y-%m-%d')))

In [35]:
stocks['Close'] = data.Data[0]
stocks['MarketValue'] = stocks['SharesHolding'] * stocks['Close']

In [36]:
stocks = stocks.sort_values('RatioInNV', ascending=False)

In [37]:
stocks.head()

,,SharesHolding,ChangeOfSharesHolding,RatioInNV,WindCode,Close,MarketValue
StockCode,StockAbbr,,,,,,
002925,盈趣科技,2731468.0,-301800.0,0.0999,002925.SZ,58.25,1.591080e+08
002371,北方华创,9857930.0,614379.0,0.0960,002371.SZ,157.20,1.549667e+09
300782,卓胜微,3587978.0,1641020.0,0.0903,300782.SZ,341.86,1.226586e+09
688029,南微医学,2124370.0,537935.0,0.0892,688029.SH,210.38,4.469250e+08
603986,兆易创新,6062488.0,1702656.0,0.0887,603986.SH,178.90,1.084579e+09


# 主题行业分类

In [38]:
industry = pd.read_excel('../data/申万行业分类表.xlsx')

In [39]:
industry2theme = {}
for col in industry.columns:
    for i in industry[col]:
        if type(i) == str:
            industry2theme[i.rstrip('(申万)')] = col
            industry2theme[i.rstrip('Ⅱ(申万)')] = col
#             print(i.rstrip('Ⅱ(申万)'), col)
industry2theme['证券Ⅱ'] = '金融地产'

In [40]:
data = w.wss(stocks['WindCode'].tolist(), 'industry_sw', 'industryType=2')

In [41]:
stocks['industry_sw'] = data.Data[0]

In [42]:
stocks['industry_theme'] = [industry2theme[x] for x in stocks['industry_sw']]

In [100]:
del stocks['WindCode']

In [102]:
stocks.head()

,,SharesHolding,ChangeOfSharesHolding,RatioInNV,Close,MarketValue,industry_sw,industry_theme
StockCode,StockAbbr,,,,,,,
002925,盈趣科技,5434232.0,-264473.0,0.09655,65.50,3.559422e+08,电子制造,信息科技
002371,北方华创,9857930.0,614379.0,0.09600,187.00,1.843433e+09,半导体,信息科技
300782,卓胜微,3587978.0,1641020.0,0.09030,450.39,1.615989e+09,半导体,信息科技
688029,南微医学,2124370.0,537935.0,0.08920,230.20,4.890300e+08,医疗器械Ⅱ,必选医药
603986,兆易创新,6062488.0,1702656.0,0.08870,242.53,1.470335e+09,半导体,信息科技


In [99]:
stocks.to_excel('../data/基金池基金重仓股.xlsx')